# Task 2

## 1) Normalize the data-set

## 2) Build a LSTM architecture to classify this data-set

GIT: https://github.com/RafaelDrumond/PeekDB

Run for 25 epochs and report your results.
Classes used:  "Crouching", "Running", "Swing"

Use Actor 2 for validation, and actor 3 and onwards for training. Remember to use np.array to transform the list into numpy array.

Pay attention to:
  The size of the window (use 3 different sizes)

## 3) On the current state the loaded data-set is not optimal, you must improve the loading function (hint: the git points to a certain paper that might help with and augmentation)

## 4) BONUS: use early time-series to see if you can improve the scores

# Loading Data
Loading the dataset is done through github. By using the host's content network, no local storage was necessary

In [1]:
import tensorflow as tf
import numpy as np
import urllib
from time import time

files = ["Actor0Crouching.txt", "Actor0Idle.txt", "Actor0Running.txt", "Actor0Swing.txt", "Actor0Walking.txt", "Actor10Crouching.txt", "Actor10Idle.txt", "Actor10Running.txt", "Actor10Swing.txt", "Actor10Walking.txt", "Actor1Crouching.txt", "Actor1Idle.txt", "Actor1Running.txt", "Actor1Swing.txt", "Actor1Walking.txt", "Actor2Crouching.txt", "Actor2Idle.txt", "Actor2Running.txt", "Actor2Swing.txt", "Actor2Walking.txt", "Actor3Crouching.txt", "Actor3Idle.txt", "Actor3Running.txt", "Actor3Swing.txt", "Actor3Walking.txt", "Actor4Crouching.txt", "Actor4Idle.txt", "Actor4Running.txt", "Actor4Swing.txt", "Actor4Walking.txt", "Actor5Crouching.txt", "Actor5Idle.txt", "Actor5Running.txt", "Actor5Swing.txt", "Actor5Walking.txt", "Actor6Crouching.txt", "Actor6Idle.txt", "Actor6Running.txt", "Actor6Swing.txt", "Actor6Walking.txt", "Actor7Crouching.txt", "Actor7Idle.txt", "Actor7Running.txt", "Actor7Swing.txt", "Actor7Walking.txt", "Actor8Crouching.txt", "Actor8Idle.txt", "Actor8Running.txt", "Actor8Swing.txt", "Actor8Walking.txt", "Actor9Crouching.txt", "Actor9Idle.txt", "Actor9Running.txt", "Actor9Swing.txt", "Actor9Walking.txt"]
base_url = "https://raw.githubusercontent.com/RafaelDrumond/PeekDB/master/Motion/"

classes = ["Crouching", "Idle", "Running", "Siwing", "Walking"]

Selecting the window size and labels to be used in the model:

In [2]:
actions = classes
window  = 60
# window  = 40
# window  = 20

A helper function to build the data url's based on the actor and action

In [3]:
def build_url(actor, action):
    assert action in classes
    assert actor in range(11)
    return base_url+"Actor"+str(actor)+action+".txt"

Load the training data:

In [4]:
actors = range(3,11)
n_actions = len(actions)
xt = np.zeros((len(actors)*n_actions,20,window))
yt = np.zeros((len(actors)*n_actions,n_actions))
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yt[action, :]        = class_vector
    print(action_name, class_vector)
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xt[actor*n_actions+action, :, i] = line
        
print(xt.shape, yt.shape)

Crouching [1, 0, 0, 0, 0]
Idle [0, 1, 0, 0, 0]
Running [0, 0, 1, 0, 0]
Siwing [0, 0, 0, 1, 0]
Walking [0, 0, 0, 0, 1]
(40, 20, 60) (40, 5)


Load the validation data:

In [5]:
actors = [2]
n_actions = len(actions)
xv = np.zeros((len(actors)*n_actions, 20, window)) # event, features, time
yv = np.zeros((len(actors)*n_actions, n_actions))
print(xv.shape, yv.shape)
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yv[action, :]        = class_vector
    print(action_name, class_vector)
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xv[actor*n_actions+action, :, i] = line
        
print(xv.shape, yv.shape)

(5, 20, 60) (5, 5)
Crouching [1, 0, 0, 0, 0]
Idle [0, 1, 0, 0, 0]
Running [0, 0, 1, 0, 0]
Siwing [0, 0, 0, 1, 0]
Walking [0, 0, 0, 0, 1]
(5, 20, 60) (5, 5)


In [6]:
x = tf.placeholder(tf.float32, [None, 20, window], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.keras.layers.LSTM(200, name="LSTM1")(x)
model = tf.layers.Dropout(.5, name="dropout1")(model)
model = tf.layers.Dense(27, name="dense1")(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")


epochs = 200
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        sess.run(optimizer,feed_dict={x: xt, y: yt})
        vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
        tloss, tacc = sess.run([loss, accuracy], feed_dict={x: xt, y: yt})
        print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    print("Training time:", time() - tic)


Instructions for updating:
Colocations handled automatically by placer.
Begin Session
Start Epoch  1
Validation Loss:   2.0851, Validation Accuracy: 0.000000
Training   Loss:   0.1325, Training   Accuracy: 0.300000
Epoch time:  0.37860894203186035
Start Epoch  2
Validation Loss:   1.9281, Validation Accuracy: 0.200000
Training   Loss:   0.0992, Training   Accuracy: 0.400000
Epoch time:  0.03984951972961426
Start Epoch  3
Validation Loss:   1.8737, Validation Accuracy: 0.200000
Training   Loss:   0.0872, Training   Accuracy: 0.400000
Epoch time:  0.027487754821777344
Start Epoch  4
Validation Loss:   1.8411, Validation Accuracy: 0.200000
Training   Loss:   0.0803, Training   Accuracy: 0.400000
Epoch time:  0.026973247528076172
Start Epoch  5
Validation Loss:   1.8423, Validation Accuracy: 0.200000
Training   Loss:   0.0752, Training   Accuracy: 0.425000
Epoch time:  0.02910304069519043
Start Epoch  6
Validation Loss:   1.8286, Validation Accuracy: 0.200000
Training   Loss:   0.0698, Tra

Validation Loss:   1.6606, Validation Accuracy: 0.200000
Training   Loss:   0.0437, Training   Accuracy: 0.225000
Epoch time:  0.02465534210205078
Start Epoch 58
Validation Loss:   1.6549, Validation Accuracy: 0.200000
Training   Loss:   0.0424, Training   Accuracy: 0.225000
Epoch time:  0.026021242141723633
Start Epoch 59
Validation Loss:   1.6495, Validation Accuracy: 0.200000
Training   Loss:   0.0410, Training   Accuracy: 0.225000
Epoch time:  0.024369239807128906
Start Epoch 60
Validation Loss:   1.6599, Validation Accuracy: 0.200000
Training   Loss:   0.0388, Training   Accuracy: 0.225000
Epoch time:  0.02422809600830078
Start Epoch 61
Validation Loss:   1.6741, Validation Accuracy: 0.200000
Training   Loss:   0.0373, Training   Accuracy: 0.200000
Epoch time:  0.024561405181884766
Start Epoch 62
Validation Loss:   1.6772, Validation Accuracy: 0.200000
Training   Loss:   0.0367, Training   Accuracy: 0.200000
Epoch time:  0.024998188018798828
Start Epoch 63
Validation Loss:   1.643

Validation Loss:   1.6545, Validation Accuracy: 0.200000
Training   Loss:   0.0343, Training   Accuracy: 0.200000
Epoch time:  0.026473045349121094
Start Epoch 113
Validation Loss:   1.6690, Validation Accuracy: 0.200000
Training   Loss:   0.0342, Training   Accuracy: 0.200000
Epoch time:  0.024845361709594727
Start Epoch 114
Validation Loss:   1.6779, Validation Accuracy: 0.200000
Training   Loss:   0.0336, Training   Accuracy: 0.200000
Epoch time:  0.02462005615234375
Start Epoch 115
Validation Loss:   1.6728, Validation Accuracy: 0.200000
Training   Loss:   0.0331, Training   Accuracy: 0.200000
Epoch time:  0.02526688575744629
Start Epoch 116
Validation Loss:   1.6573, Validation Accuracy: 0.200000
Training   Loss:   0.0327, Training   Accuracy: 0.200000
Epoch time:  0.024786710739135742
Start Epoch 117
Validation Loss:   1.6553, Validation Accuracy: 0.200000
Training   Loss:   0.0324, Training   Accuracy: 0.175000
Epoch time:  0.025199174880981445
Start Epoch 118
Validation Loss:  

Validation Loss:   1.6320, Validation Accuracy: 0.200000
Training   Loss:   0.0321, Training   Accuracy: 0.150000
Epoch time:  0.025610685348510742
Start Epoch 169
Validation Loss:   1.6454, Validation Accuracy: 0.200000
Training   Loss:   0.0322, Training   Accuracy: 0.150000
Epoch time:  0.025220870971679688
Start Epoch 170
Validation Loss:   1.6553, Validation Accuracy: 0.200000
Training   Loss:   0.0324, Training   Accuracy: 0.175000
Epoch time:  0.026659488677978516
Start Epoch 171
Validation Loss:   1.6815, Validation Accuracy: 0.200000
Training   Loss:   0.0326, Training   Accuracy: 0.175000
Epoch time:  0.02473926544189453
Start Epoch 172
Validation Loss:   1.6748, Validation Accuracy: 0.200000
Training   Loss:   0.0333, Training   Accuracy: 0.175000
Epoch time:  0.03138160705566406
Start Epoch 173
Validation Loss:   1.6839, Validation Accuracy: 0.200000
Training   Loss:   0.0341, Training   Accuracy: 0.175000
Epoch time:  0.042198896408081055
Start Epoch 174
Validation Loss:  

Different window sizes:

In [7]:
import tensorflow as tf
import numpy as np
import urllib
from time import time

files = ["Actor0Crouching.txt", "Actor0Idle.txt", "Actor0Running.txt", "Actor0Swing.txt", "Actor0Walking.txt", "Actor10Crouching.txt", "Actor10Idle.txt", "Actor10Running.txt", "Actor10Swing.txt", "Actor10Walking.txt", "Actor1Crouching.txt", "Actor1Idle.txt", "Actor1Running.txt", "Actor1Swing.txt", "Actor1Walking.txt", "Actor2Crouching.txt", "Actor2Idle.txt", "Actor2Running.txt", "Actor2Swing.txt", "Actor2Walking.txt", "Actor3Crouching.txt", "Actor3Idle.txt", "Actor3Running.txt", "Actor3Swing.txt", "Actor3Walking.txt", "Actor4Crouching.txt", "Actor4Idle.txt", "Actor4Running.txt", "Actor4Swing.txt", "Actor4Walking.txt", "Actor5Crouching.txt", "Actor5Idle.txt", "Actor5Running.txt", "Actor5Swing.txt", "Actor5Walking.txt", "Actor6Crouching.txt", "Actor6Idle.txt", "Actor6Running.txt", "Actor6Swing.txt", "Actor6Walking.txt", "Actor7Crouching.txt", "Actor7Idle.txt", "Actor7Running.txt", "Actor7Swing.txt", "Actor7Walking.txt", "Actor8Crouching.txt", "Actor8Idle.txt", "Actor8Running.txt", "Actor8Swing.txt", "Actor8Walking.txt", "Actor9Crouching.txt", "Actor9Idle.txt", "Actor9Running.txt", "Actor9Swing.txt", "Actor9Walking.txt"]
base_url = "https://raw.githubusercontent.com/RafaelDrumond/PeekDB/master/Motion/"

classes = ["Crouching", "Idle", "Running", "Siwing", "Walking"]

actions = classes
window  = 40

def build_url(actor, action):
    assert action in classes
    assert actor in range(11)
    return base_url+"Actor"+str(actor)+action+".txt"


actors = range(3,11)
n_actions = len(actions)
xt = np.zeros((len(actors)*n_actions,20,window))
yt = np.zeros((len(actors)*n_actions,n_actions))
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yt[action, :]        = class_vector
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xt[actor*n_actions+action, :, i] = line

actors = [2]
n_actions = len(actions)
xv = np.zeros((len(actors)*n_actions, 20, window)) # event, features, time
yv = np.zeros((len(actors)*n_actions, n_actions))
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yv[action, :]        = class_vector
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xv[actor*n_actions+action, :, i] = line

                
x = tf.placeholder(tf.float32, [None, 20, window], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.keras.layers.LSTM(200, name="LSTM1")(x)
model = tf.layers.Dropout(.5, name="dropout1")(model)
model = tf.layers.Dense(27, name="dense1")(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")


epochs = 50
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        sess.run(optimizer,feed_dict={x: xt, y: yt})
        vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
        tloss, tacc = sess.run([loss, accuracy], feed_dict={x: xt, y: yt})
        print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    print("Training time:", time() - tic)

Begin Session
Start Epoch  1
Validation Loss:   1.6569, Validation Accuracy: 0.400000
Training   Loss:   0.1385, Training   Accuracy: 0.175000
Epoch time:  0.24529409408569336
Start Epoch  2
Validation Loss:   1.6801, Validation Accuracy: 0.400000
Training   Loss:   0.1131, Training   Accuracy: 0.150000
Epoch time:  0.02607560157775879
Start Epoch  3
Validation Loss:   1.5518, Validation Accuracy: 0.600000
Training   Loss:   0.0915, Training   Accuracy: 0.125000
Epoch time:  0.024786949157714844
Start Epoch  4
Validation Loss:   1.4996, Validation Accuracy: 0.800000
Training   Loss:   0.0791, Training   Accuracy: 0.125000
Epoch time:  0.025250673294067383
Start Epoch  5
Validation Loss:   1.5203, Validation Accuracy: 0.800000
Training   Loss:   0.0712, Training   Accuracy: 0.125000
Epoch time:  0.024490833282470703
Start Epoch  6
Validation Loss:   1.5151, Validation Accuracy: 0.800000
Training   Loss:   0.0661, Training   Accuracy: 0.125000
Epoch time:  0.024782419204711914
Start Epoc

In [8]:
import tensorflow as tf
import numpy as np
import urllib
from time import time

files = ["Actor0Crouching.txt", "Actor0Idle.txt", "Actor0Running.txt", "Actor0Swing.txt", "Actor0Walking.txt", "Actor10Crouching.txt", "Actor10Idle.txt", "Actor10Running.txt", "Actor10Swing.txt", "Actor10Walking.txt", "Actor1Crouching.txt", "Actor1Idle.txt", "Actor1Running.txt", "Actor1Swing.txt", "Actor1Walking.txt", "Actor2Crouching.txt", "Actor2Idle.txt", "Actor2Running.txt", "Actor2Swing.txt", "Actor2Walking.txt", "Actor3Crouching.txt", "Actor3Idle.txt", "Actor3Running.txt", "Actor3Swing.txt", "Actor3Walking.txt", "Actor4Crouching.txt", "Actor4Idle.txt", "Actor4Running.txt", "Actor4Swing.txt", "Actor4Walking.txt", "Actor5Crouching.txt", "Actor5Idle.txt", "Actor5Running.txt", "Actor5Swing.txt", "Actor5Walking.txt", "Actor6Crouching.txt", "Actor6Idle.txt", "Actor6Running.txt", "Actor6Swing.txt", "Actor6Walking.txt", "Actor7Crouching.txt", "Actor7Idle.txt", "Actor7Running.txt", "Actor7Swing.txt", "Actor7Walking.txt", "Actor8Crouching.txt", "Actor8Idle.txt", "Actor8Running.txt", "Actor8Swing.txt", "Actor8Walking.txt", "Actor9Crouching.txt", "Actor9Idle.txt", "Actor9Running.txt", "Actor9Swing.txt", "Actor9Walking.txt"]
base_url = "https://raw.githubusercontent.com/RafaelDrumond/PeekDB/master/Motion/"

classes = ["Crouching", "Idle", "Running", "Siwing", "Walking"]

actions = classes
window  = 20

def build_url(actor, action):
    assert action in classes
    assert actor in range(11)
    return base_url+"Actor"+str(actor)+action+".txt"


actors = range(3,11)
n_actions = len(actions)
xt = np.zeros((len(actors)*n_actions,20,window))
yt = np.zeros((len(actors)*n_actions,n_actions))
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yt[action, :]        = class_vector
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xt[actor*n_actions+action, :, i] = line

actors = [2]
n_actions = len(actions)
xv = np.zeros((len(actors)*n_actions, 20, window)) # event, features, time
yv = np.zeros((len(actors)*n_actions, n_actions))
for action, action_name in enumerate(actions):
    class_vector         = [0] * len(actions)
    class_vector[action] = 1
    yv[action, :]        = class_vector
    for actor, actor_n in enumerate(actors):
        url = build_url(actor_n, action_name)
        if url in [base_url+f for f in files]:
            response = urllib.request.urlopen(url)
            response = response.read().decode("utf8")
            response = response.split("\r\n")
            for i,line in enumerate(response[0:window]):
                line = line.split("|m ")[1].split(" |class")[0].split(" ")
                xv[actor*n_actions+action, :, i] = line

                
x = tf.placeholder(tf.float32, [None, 20, window], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.keras.layers.LSTM(200, name="LSTM1")(x)
model = tf.layers.Dropout(.5, name="dropout1")(model)
model = tf.layers.Dense(27, name="dense1")(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")


epochs = 50
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        sess.run(optimizer,feed_dict={x: xt, y: yt})
        vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
        tloss, tacc = sess.run([loss, accuracy], feed_dict={x: xt, y: yt})
        print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    print("Training time:", time() - tic)

Begin Session
Start Epoch  1
Validation Loss:   1.3657, Validation Accuracy: 0.400000
Training   Loss:   0.1371, Training   Accuracy: 0.100000
Epoch time:  0.2590920925140381
Start Epoch  2
Validation Loss:   1.2963, Validation Accuracy: 0.400000
Training   Loss:   0.1042, Training   Accuracy: 0.125000
Epoch time:  0.02444911003112793
Start Epoch  3
Validation Loss:   1.2585, Validation Accuracy: 0.400000
Training   Loss:   0.0910, Training   Accuracy: 0.175000
Epoch time:  0.02443242073059082
Start Epoch  4
Validation Loss:   1.2198, Validation Accuracy: 0.400000
Training   Loss:   0.0852, Training   Accuracy: 0.175000
Epoch time:  0.024080514907836914
Start Epoch  5
Validation Loss:   1.1981, Validation Accuracy: 0.400000
Training   Loss:   0.0813, Training   Accuracy: 0.200000
Epoch time:  0.025050640106201172
Start Epoch  6
Validation Loss:   1.2007, Validation Accuracy: 0.400000
Training   Loss:   0.0782, Training   Accuracy: 0.200000
Epoch time:  0.023611783981323242
Start Epoch 